# Chapter 8 exercises

In [1]:
import pandas as pd
import numpy as np

### 8.3

Consider three binary variables $a, b, c \in \{0, 1\}$ having the joint distribution given by the table below. Show by direct evaluation that this distribution has the property that $a$ and $b$s are marginally dependent, so that $p(a,b) \neq p(a)p(b)$, but that they become independent when conditioned on $c$ so that $p(a,b|c) = p(a|c)p(b|c)$ fo both $c=0$ and $c=1$

In [38]:
p = np.array([
    [0, 0, 0, 0.192],
    [0, 0, 1, 0.144],
    [0, 1, 0, 0.048],
    [0, 1, 1, 0.216],
    [1, 0, 0, 0.192],
    [1, 0, 1, 0.064],
    [1, 1, 0, 0.048],
    [1, 1, 1, 0.096],
])

p = pd.DataFrame(p, columns=["a", "b", "c", "p_abc"])
p = p.set_index(["a", "b", "c"])

Marginalization without conditioning

In [42]:
# p(a, b)
p.xs(0, level=2) + p.xs(1, level=2)

p_abc
a   b         
0.0 0.0  0.336
    1.0  0.264
1.0 0.0  0.256
    1.0  0.144

In [161]:
# p(a)p(b)
p_marg = p.xs(0, level=2) + p.xs(1, level=2)
p_a = p_marg.xs(0, level=-1) + p_marg.xs(1, level=-1)
p_b = p_marg.xs(0, level=0) + p_marg.xs(1, level=0)

p_a["key"] = 1
p_b["key"] = 1

p_marg = pd.merge(p_a.reset_index(), p_b.reset_index(), on="key",
         suffixes=("_a", "_b")).drop("key", axis=1)

p_marg.set_index(["a", "b"]).prod(axis=1)

a    b  
0.0  0.0    0.3552
     1.0    0.2448
1.0  0.0    0.2368
     1.0    0.1632
dtype: float64

Factorization with conditioning

In [166]:
# p(a,b|c=0)
p_marg = p.xs(0, level=-1)
p_marg

p_abc
a   b         
0.0 0.0  0.192
    1.0  0.048
1.0 0.0  0.192
    1.0  0.048

In [151]:
# p(a| c=0)
p_a = p_marg.sum(level=0)
p_b = p_marg.sum(level=1)

p_a = p_a.reset_index().assign(key=1)
p_b = p_b.reset_index().assign(key=1)

p_fact = (pd.merge(p_a, p_b, on="key", suffixes=("_a", "_b"))
            .drop("key", axis=1)
            .set_index(["a", "b"]))

In [154]:
p_fact.prod(axis=1)

a    b  
0.0  0.0    0.07488
     1.0    0.11232
1.0  0.0    0.03328
     1.0    0.04992
dtype: float64

In [119]:
# p(a,b|c=0)
p_marg = p.xs(0, level=-1)
p_marg.sum(level=1)

,p_abc
b,
0.0,0.384
1.0,0.096
